In [1]:
%pip install pandas numpy seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


In [3]:
print('Lib versions:')
print(f'Pandas: {pd.__version__}')
print(f'Numpy: {np.__version__}')
print(f'Seaborn: {sns.__version__}')
print('\nPython version:')
!python3.12 --version

Lib versions:
Pandas: 2.2.3
Numpy: 2.1.1
Seaborn: 0.13.2

Python version:
Python 3.12.6


In [15]:
!wget -P data https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-10-14 16:15:00--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/bank+marketing.zip’

bank+marketing.zip      [       <=>          ] 999.85K   810KB/s    in 1.2s    

2024-10-14 16:15:02 (810 KB/s) - ‘data/bank+marketing.zip’ saved [1023843]



In [16]:
!unzip data/bank+marketing.zip -d data

Archive:  data/bank+marketing.zip
 extracting: data/bank.zip           
 extracting: data/bank-additional.zip  


In [17]:
!unzip data/bank.zip -d data
!unzip data/bank-additional.zip -d data

Archive:  data/bank.zip
  inflating: data/bank-full.csv      
  inflating: data/bank-names.txt     
  inflating: data/bank.csv           
Archive:  data/bank-additional.zip
   creating: data/bank-additional/
  inflating: data/bank-additional/.DS_Store  
   creating: data/__MACOSX/
   creating: data/__MACOSX/bank-additional/
  inflating: data/__MACOSX/bank-additional/._.DS_Store  
  inflating: data/bank-additional/.Rhistory  
  inflating: data/bank-additional/bank-additional-full.csv  
  inflating: data/bank-additional/bank-additional-names.txt  
  inflating: data/bank-additional/bank-additional.csv  
  inflating: data/__MACOSX/._bank-additional  


In [18]:
!rm data/bank-additional.zip data/bank.zip data/bank+marketing.zip

# Loading and preparing the dataset

In [20]:
df = pd.read_csv('./data/bank-full.csv', delimiter=';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


### Features
For the rest of the homework, you'll need to use only these columns:

age,
job,
marital,
education,
balance,
housing,
contact,
day,
month,
duration,
campaign,
pdays,
previous,
poutcome,
y

In [22]:
df = df[['age',
'job',
'marital',
'education',
'balance',
'housing',
'contact',
'day',
'month',
'duration',
'campaign',
'pdays',
'previous',
'poutcome',
'y']]
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


### Checking missing values

In [45]:
null_cols = df.columns[df.isna().any()].tolist()
print(f'Null columns: {null_cols}')

Null columns: []


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   balance    45211 non-null  int64 
 5   housing    45211 non-null  object
 6   contact    45211 non-null  object
 7   day        45211 non-null  int64 
 8   month      45211 non-null  object
 9   duration   45211 non-null  int64 
 10  campaign   45211 non-null  int64 
 11  pdays      45211 non-null  int64 
 12  previous   45211 non-null  int64 
 13  poutcome   45211 non-null  object
 14  y          45211 non-null  object
dtypes: int64(7), object(8)
memory usage: 5.2+ MB


In [43]:
string_cols = df.columns[df.dtypes == 'object']

print('String values: ')
df[string_cols].apply(lambda x: print(f'\t{x.name}: {x.unique()}\n'))
print()

String values: 
	job: ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']

	marital: ['married' 'single' 'divorced']

	education: ['tertiary' 'secondary' 'unknown' 'primary']

	housing: ['yes' 'no']

	contact: ['unknown' 'cellular' 'telephone']

	month: ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']

	poutcome: ['unknown' 'failure' 'other' 'success']

	y: ['no' 'yes']




There's no `'nan'` or `''` values

### Q1. Mode for the column `education`


In [47]:
df.education.mode()

0    secondary
Name: education, dtype: object

### Q2. Two features with the biggest correlation


In [53]:
correlation_matrix = df.select_dtypes(include='int64').corr()
correlation_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


In [55]:
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

max_corr = upper_triangle.unstack().abs().sort_values(ascending=False).head(1)

print("Features with the biggest correlation:", max_corr)

Features with the biggest correlation: previous  pdays    0.45482
dtype: float64


### Target encoding + Spliting the dataset

- Now we want to encode the y variable.
- Let's replace the values yes/no with 1/0.
- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value y is not in your dataframe.

In [56]:
df['y'] = df['y'].replace({'yes': 1, 'no': 0})
df['y'].unique()

/tmp/ipykernel_2347/579347080.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['y'] = df['y'].replace({'yes': 1, 'no': 0})
/tmp/ipykernel_2347/579347080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = df['y'].replace({'yes': 1, 'no': 0})


array([0, 1])

In [57]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['y'])
y = df['y']

# 60% train + 40% temp
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
# 50% validation and 50% test = 20% val + 20% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train: {X_train.shape[0]}")
print(f"Validation: {X_val.shape[0]}")
print(f"Test: {X_test.shape[0]}")

Train: 27126
Validation: 9042
Test: 9043


### Q3. Variable with the biggest mutual information score

- Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

In [85]:
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns
X_train_categorical = X_train[categorical_columns]
X_train_categorical

,job,marital,education,housing,contact,month,poutcome
6377,entrepreneur,married,primary,yes,unknown,may,unknown
17236,services,single,secondary,no,cellular,jul,unknown
4490,admin.,single,secondary,yes,unknown,may,unknown
24231,self-employed,single,tertiary,yes,cellular,nov,unknown
3978,services,single,secondary,yes,unknown,may,unknown
...,...,...,...,...,...,...,...
11284,housemaid,single,primary,no,unknown,jun,unknown
44732,student,single,tertiary,no,cellular,sep,failure
38158,technician,divorced,tertiary,yes,cellular,may,unknown
860,retired,married,secondary,no,unknown,may,unknown


In [86]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X_train_categorical.loc[:, col] = le.fit_transform(X_train_categorical[col])
    label_encoders[:, col] = le

mi_scores = mutual_info_classif(X_train_categorical, y_train, discrete_features=True)
mi_scores_rounded = [round(score, 2) for score in mi_scores]
mi_scores_rounded

[np.float64(0.01),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.01),
 np.float64(0.01),
 np.float64(0.02),
 np.float64(0.03)]

In [88]:
mi_df = pd.DataFrame({'Variable': categorical_columns, 'MI Score': mi_scores_rounded})

mi_df_sorted = mi_df.sort_values(by='MI Score', ascending=False)

display(mi_df_sorted)

print("Highest mutual information score:", mi_df_sorted.iloc[0])

,Variable,MI Score
6,poutcome,0.03
5,month,0.02
0,job,0.01
4,contact,0.01
3,housing,0.01
2,education,0.00
1,marital,0.00


Highest mutual information score: Variable    poutcome
MI Score        0.03
Name: 6, dtype: object


### Q4. Accuracy on the validation dataset 

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [102]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [100]:
X_train_dict = X_train.to_dict(orient='records')
X_val_dict = X_val.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train_encoded = dv.fit_transform(X_train_dict)

X_val_encoded = dv.transform(X_val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [147]:
y_pred = model.predict(X_val_encoded)

accuracy = accuracy_score(y_val, y_pred)

print(f'Accuracy on validation: {round(accuracy, 2)}')

Accuracy on validation: 0.9


### Q5. Feature with the smallest difference

- Let's find the least useful feature using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?
- age
- balance
- marital
- previous
> Note: The difference doesn't have to be positive.

In [131]:
def train_and_evaluate(X_train, X_val, y_train, y_val):
    X_train_dict = X_train.to_dict(orient='records')
    X_val_dict = X_val.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train_encoded = dv.fit_transform(X_train_dict)
    X_val_encoded = dv.transform(X_val_dict)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_encoded, y_train)

    y_pred = model.predict(X_val_encoded)
    return accuracy_score(y_val, y_pred)

In [135]:
accuracy_full = train_and_evaluate(X_train, X_val, y_train, y_val)
print(f'Accuracy with all features: {accuracy_full}')

Accuracy with all features: 0.9007962840079629


In [139]:
features_to_remove = ['age', 'balance', 'marital', 'previous']
differences = {}

for feature in features_to_remove:
    X_train_reduced = X_train.drop(columns=[feature])
    X_val_reduced = X_val.drop(columns=[feature])

    accuracy_without_feature = train_and_evaluate(X_train_reduced, X_val_reduced, y_train, y_val)
    
    difference = abs(accuracy_full - accuracy_without_feature)
    differences[feature] = round(difference, 4) 

    print(f'Accuracy without {feature}: {accuracy_without_feature} (Difference: {difference})')

Accuracy without age: 0.9010174740101747 (Difference: 0.00022119000221187957)
Accuracy without balance: 0.9007962840079629 (Difference: 0.0)
Accuracy without marital: 0.9011280690112807 (Difference: 0.00033178500331787486)
Accuracy without previous: 0.9006856890068569 (Difference: 0.0001105950011059953)


In [141]:
least_impact_feature = min(differences, key=lambda x: differences[x])
print(f'Feature with the smallest difference: {least_impact_feature}')

Feature with the smallest difference: balance


### Q6. Smallest `C` that leads to the best accuracy on the validation set

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

In [144]:
def train_and_evaluate_reg(X_train, X_val, y_train, y_val, C_value):
    X_train_dict = X_train.to_dict(orient='records')
    X_val_dict = X_val.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train_encoded = dv.fit_transform(X_train_dict)
    X_val_encoded = dv.transform(X_val_dict)

    model = LogisticRegression(solver='liblinear', C=C_value, max_iter=1000, random_state=42)
    model.fit(X_train_encoded, y_train)

    y_pred = model.predict(X_val_encoded)
    return round(accuracy_score(y_val, y_pred), 3)

In [146]:
C_values = [0.01, 0.1, 1, 10, 100]
best_accuracy = 0
best_C = None

for C in C_values:
    accuracy = train_and_evaluate_reg(X_train, X_val, y_train, y_val, C)
    print(f'Acurácia com C={C}: {accuracy}')
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C

print(f'Best value of C: {best_C}, with {best_accuracy} of accuracy.')

Acurácia com C=0.01: 0.899
Acurácia com C=0.1: 0.901
Acurácia com C=1: 0.901
Acurácia com C=10: 0.901
Acurácia com C=100: 0.901
Best value of C: 0.1, with 0.901 of accuracy.
